In [1]:
import mysql.connector
import json
from datetime import datetime

# Function to create CVE tables for each year
def create_database_schema():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="sn00ky17",
            database="CVES13"
        )
        cursor = conn.cursor()

        # Schema for CVE records 2020<
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS CVE_2020_before (
                CVE_ID VARCHAR(255) PRIMARY KEY,
                Description TEXT,
                Published_Date DATE,
                Last_Modified_Date DATE,
                CVSS VARCHAR(50),
                CWE VARCHAR(50),
                URL VARCHAR(1000)             
            )
        """)

        # Schema for CVE records 2021-2024
        for year in range(2021, 2025):
            table_name = f"CVE_{year}"
            cursor.execute(f"""
                CREATE TABLE IF NOT EXISTS {table_name} (
                    CVE_ID VARCHAR(255) PRIMARY KEY,
                    Description TEXT,
                    Published_Date DATE,
                    Last_Modified_Date DATE,
                    CVSS VARCHAR(50),
                    CWE VARCHAR(50),
                    URL VARCHAR(1000)
                )
            """)

        conn.commit()
        conn.close()
        print("Database schema created successfully")
    except mysql.connector.Error as error:
        print("Error while creating database schema:", error)

# Parsing through .json file and importing data into database
def parse_and_import_cve_records(file_path):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="sn00ky17",
        database="CVES13"
    )
    cursor = conn.cursor()

    count = 0  # Counter variable to import specific num of records

    with open(file_path, 'r') as file:
        for line in file:
            try:
                if count >= 1000:
                    break  # Loop break

                cve_data = json.loads(line)

                cve_id = cve_data.get("id")
                description = cve_data.get("summary")
                published_date = datetime.strptime(cve_data.get("Published"), "%Y-%m-%d %H:%M:%S")
                last_modified_date = datetime.strptime(cve_data.get("Modified"), "%Y-%m-%d %H:%M:%S")
                CVSS = cve_data.get("cvss")
                cwe = cve_data.get("cwe")
                
                references = cve_data.get("references")
                url = references[0] if references else None

                # Determine the appropriate table based on the CVE publication year
                if published_date.year <= 2020:
                    table_name = "CVE_2020_before"
                else:
                    table_name = f"CVE_{published_date.year}"

                # Insert or update CVE record
                cursor.execute(f"""
                    INSERT INTO {table_name} (CVE_ID, Description, Published_Date, Last_Modified_Date, CVSS, CWE, URL)
                    VALUES (%s, %s, %s, %s, %s, %s, %s)
                    ON DUPLICATE KEY UPDATE
                    Description=VALUES(Description),
                    Published_Date=VALUES(Published_Date),
                    Last_Modified_Date=VALUES(Last_Modified_Date),
                    CVSS=VALUES(CVSS),
                    CWE=VALUES(CWE),
                    URL=VALUES(URL)
                """, (cve_id, description, published_date, last_modified_date, CVSS, cwe, url))

                count += 1  # Increment for count

            except mysql.connector.Error as error:
                print("Error while parsing and importing CVE records:", error)

    conn.commit()
    conn.close()

In [ ]:
# Main function
def main():
    # Creating schema
    create_database_schema()

    # File path for .json data
    file_path = "./Documents/CVERecord.json"

    # Function call to parse CVE records
    parse_and_import_cve_records(file_path)

if __name__ == "__main__":
    main()